In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

# Merge datasets
merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

# Create a pivot table for customer-product purchase quantity
customer_product_matrix = pd.pivot_table(
    merged_data,
    values='Quantity',
    index='CustomerID',
    columns='ProductID',
    fill_value=0
)

# Compute cosine similarity between customers
similarity_matrix = cosine_similarity(customer_product_matrix)

# Create a similarity DataFrame
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=customer_product_matrix.index,
    columns=customer_product_matrix.index
)

# Generate Lookalike recommendations for the first 20 customers
lookalikes = {}
for customer_id in customers['CustomerID'][:20]:  # First 20 customers
    if customer_id in similarity_df.index:
        # Get the top 3 similar customers (excluding the customer themselves)
        top_similar = similarity_df[customer_id].sort_values(ascending=False)[1:4]
        # Create a list of tuples (customer_id, similarity_score)
        lookalikes[customer_id] = list(zip(top_similar.index, top_similar.values))

# Save the results in the specified format
lookalike_df = pd.DataFrame.from_dict(lookalikes, orient='index')

# Save to a CSV file, where each row is a customer with their lookalike list and similarity scores
lookalike_df.to_csv("Lookalike.csv", header=False, index_label="CustomerID")

print("Lookalike recommendations with similarity scores have been saved to Lookalike.csv")

Lookalike recommendations with similarity scores have been saved to Lookalike.csv
